# Java Code Chunking and ChromaDB Storage

## Overview
This notebook demonstrates how to:
1. **Parse Java code** using tree-sitter (AST-based parsing)
2. **Intelligently chunk** the code while respecting semantic boundaries
3. **Store chunks** in ChromaDB with embeddings for retrieval

## Why Tree-Based Chunking?
- **Preserves code structure**: Won't split in the middle of a method or class
- **Semantic awareness**: Understands Java syntax (classes, methods, imports)
- **Better retrieval**: Each chunk is a meaningful unit of code
- **Context preservation**: Maintains relationships between code elements

## Step 1: Install Required Dependencies

We need:
- `tree-sitter`: For parsing code into Abstract Syntax Trees (AST)
- `tree-sitter-java`: Java language grammar for tree-sitter
- `chromadb`: Vector database for storing and retrieving code chunks
- `sentence-transformers`: For generating embeddings (optional, ChromaDB uses default)

In [ ]:
# Install dependencies
!pip install tree-sitter tree-sitter-java chromadb sentence-transformers -q

In [1]:
print("Hello World")

Hello World


## Step 2: Import Libraries

Import all necessary modules for:
- File operations
- AST parsing
- Database operations
- Data structures

In [8]:
import os
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
import chromadb
from chromadb.config import Settings
from tree_sitter import Language, Parser
import tree_sitter_java
from pathlib import Path
import json

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## Step 3: Define Data Structure for Code Chunks

We create a `CodeChunk` class to represent each chunk of code with:
- **content**: The actual code text
- **start_line/end_line**: Line numbers for reference
- **chunk_type**: Type of code element (class, method, import, etc.)
- **metadata**: Additional information (file path, class name, method name, etc.)

In [9]:
@dataclass
class CodeChunk:
    """Represents a chunk of code with metadata for better retrieval."""
    content: str              # The actual code
    start_line: int          # Starting line number
    end_line: int            # Ending line number
    chunk_type: str          # Type: 'class', 'method', 'import', etc.
    metadata: Dict[str, any] # Additional context
    
    def __repr__(self):
        return f"CodeChunk(type={self.chunk_type}, lines={self.start_line}-{self.end_line}, length={len(self.content)})"

print("✓ CodeChunk class defined")

✓ CodeChunk class defined


## Step 4: Create the Java Code Chunker Class

This is the core class that:
1. **Parses Java code** into an Abstract Syntax Tree (AST)
2. **Traverses the AST** to identify meaningful code blocks
3. **Creates chunks** that respect code structure
4. **Adds overlap** between chunks for context

### Configuration Parameters:
- `max_chunk_size`: Maximum characters per chunk (prevents huge chunks)
- `chunk_overlap`: Overlapping characters between chunks (maintains context)
- `min_chunk_size`: Minimum size before merging small chunks

In [16]:
class JavaCodeChunker:
    """
    Tree-based Java code chunker using tree-sitter AST parsing.
    
    This chunker intelligently splits Java code while preserving
    semantic boundaries like class and method definitions.
    """
    
    def __init__(
        self,
        max_chunk_size: int = 1000,
        chunk_overlap: int = 200,
        min_chunk_size: int = 100
    ):
        self.max_chunk_size = max_chunk_size
        self.chunk_overlap = chunk_overlap
        self.min_chunk_size = min_chunk_size
        
        # Initialize tree-sitter parser for Java
        self.parser = Parser()
        JAVA_LANGUAGE = Language(tree_sitter_java.language())
        self.parser.language = JAVA_LANGUAGE
        print("✓ JavaCodeChunker initialized with tree-sitter")
    
    def extract_node_text(self, node, source_code: bytes) -> str:
        """Extract text from a tree-sitter node."""
        return source_code[node.start_byte:node.end_byte].decode('utf-8')
    
    def get_node_line_range(self, node) -> Tuple[int, int]:
        """Get the line range for a node (1-indexed)."""
        return node.start_point[0] + 1, node.end_point[0] + 1
    
    def chunk_code(self, code: str, file_path: str = "") -> List[CodeChunk]:
        """
        Main method: Chunk Java code using AST-based analysis.
        
        Process:
        1. Parse code into AST
        2. Identify top-level structures (package, imports, classes)
        3. Split classes into methods and fields
        4. Create chunks with metadata
        5. Merge small chunks
        6. Add overlap for context
        """
        source_bytes = code.encode('utf-8')
        tree = self.parser.parse(source_bytes)
        root_node = tree.root_node
        
        chunks = []
        
        # Process each top-level node (package, imports, classes)
        for child in root_node.children:
            chunks.extend(self._process_node(child, source_bytes, file_path))
        
        # Post-processing
        chunks = self._merge_small_chunks(chunks)
        chunks = self._add_overlap(chunks, code)
        
        return chunks
    
    def _process_node(self, node, source_bytes: bytes, file_path: str) -> List[CodeChunk]:
        """Process a single AST node and return chunks."""
        chunks = []
        node_type = node.type
        
        if node_type == 'package_declaration':
            # Package declaration - keep as single chunk
            chunks.append(self._create_chunk(node, source_bytes, 'package', file_path))
        
        elif node_type == 'import_declaration':
            # Import statement - keep as single chunk
            chunks.append(self._create_chunk(node, source_bytes, 'import', file_path))
        
        elif node_type in ['class_declaration', 'interface_declaration', 'enum_declaration']:
            # Class/Interface/Enum - process in detail
            chunks.extend(self._process_class(node, source_bytes, file_path))
        
        elif node_type == 'comment':
            # Comment - keep as single chunk
            chunks.append(self._create_chunk(node, source_bytes, 'comment', file_path))
        
        return chunks
    
    def _process_class(self, class_node, source_bytes: bytes, file_path: str) -> List[CodeChunk]:
        """Process a class/interface/enum declaration into meaningful chunks."""
        chunks = []
        
        # Extract class name for metadata
        class_name = self._extract_identifier(class_node, source_bytes)
        
        # Find the class body
        body_node = None
        for child in class_node.children:
            if child.type == 'class_body':
                body_node = child
                break
        
        if not body_node:
            # No body found, return entire class
            return [self._create_chunk(
                class_node, source_bytes, 'class', file_path,
                {'class_name': class_name}
            )]
        
        # Process each member (method, field, nested class) in the class body
        for child in body_node.children:
            if child.type in ['{', '}']:
                continue  # Skip braces
            
            if child.type in ['method_declaration', 'constructor_declaration']:
                # Method or constructor - create separate chunk
                method_name = self._extract_identifier(child, source_bytes)
                chunks.append(self._create_chunk(
                    child, source_bytes, 'method', file_path,
                    {'class_name': class_name, 'method_name': method_name}
                ))
            
            elif child.type == 'field_declaration':
                # Field declaration - create chunk
                chunks.append(self._create_chunk(
                    child, source_bytes, 'field', file_path,
                    {'class_name': class_name}
                ))
            
            else:
                # Other class members
                chunks.append(self._create_chunk(
                    child, source_bytes, 'class_member', file_path,
                    {'class_name': class_name}
                ))
        
        return chunks
    
    def _create_chunk(
        self,
        node,
        source_bytes: bytes,
        chunk_type: str,
        file_path: str,
        extra_metadata: Optional[Dict] = None
    ) -> CodeChunk:
        """Create a CodeChunk from a tree-sitter node."""
        content = self.extract_node_text(node, source_bytes)
        start_line, end_line = self.get_node_line_range(node)
        
        metadata = {
            'file_path': file_path,
            'node_type': node.type,
            'length': len(content)
        }
        if extra_metadata:
            metadata.update(extra_metadata)
        
        return CodeChunk(
            content=content,
            start_line=start_line,
            end_line=end_line,
            chunk_type=chunk_type,
            metadata=metadata
        )
    
    def _extract_identifier(self, node, source_bytes: bytes) -> str:
        """Extract identifier/name from a node (class name, method name, etc.)."""
        for child in node.children:
            if child.type == 'identifier':
                return self.extract_node_text(child, source_bytes)
        return "unknown"
    
    def _merge_small_chunks(self, chunks: List[CodeChunk]) -> List[CodeChunk]:
        """Merge chunks smaller than min_chunk_size with adjacent chunks."""
        if not chunks:
            return chunks
        
        merged = []
        current_merged = None
        
        for chunk in chunks:
            if len(chunk.content) < self.min_chunk_size:
                if current_merged is None:
                    current_merged = chunk
                else:
                    # Merge with previous small chunk
                    current_merged = CodeChunk(
                        content=current_merged.content + "\n" + chunk.content,
                        start_line=current_merged.start_line,
                        end_line=chunk.end_line,
                        chunk_type='merged',
                        metadata={**current_merged.metadata, **chunk.metadata}
                    )
            else:
                # Chunk is large enough
                if current_merged:
                    merged.append(current_merged)
                    current_merged = None
                merged.append(chunk)
        
        if current_merged:
            merged.append(current_merged)
        
        return merged
    
    def _add_overlap(self, chunks: List[CodeChunk], full_code: str) -> List[CodeChunk]:
        """
        Add overlapping text between consecutive chunks.
        
        Why? This ensures that when we retrieve a chunk, we have some
        context from the previous chunk, making the code more understandable.
        """
        if len(chunks) <= 1 or self.chunk_overlap == 0:
            return chunks
        
        overlapped = [chunks[0]]  # First chunk has no overlap
        
        for i in range(1, len(chunks)):
            chunk = chunks[i]
            prev_chunk = chunks[i - 1]
            
            # Take last N characters from previous chunk
            overlap_text = prev_chunk.content[-self.chunk_overlap:]
            
            # Create new chunk with overlap prepended
            new_chunk = CodeChunk(
                content=overlap_text + "\n" + chunk.content,
                start_line=chunk.start_line,
                end_line=chunk.end_line,
                chunk_type=chunk.chunk_type,
                metadata={**chunk.metadata, 'has_overlap': True}
            )
            overlapped.append(new_chunk)
        
        return overlapped

print("✓ JavaCodeChunker class defined")

✓ JavaCodeChunker class defined


## Step 5: Test the Chunker with Sample Java Code

Let's create a sample Java file to demonstrate the chunking process.
This example includes:
- Package declaration
- Imports
- A class with multiple methods
- Fields
- Comments

In [17]:
# Create a sample Java file
sample_java_code = """
package com.example.app;

import java.util.List;
import java.util.ArrayList;
import java.util.HashMap;

/**
 * UserManager class handles user operations
 */
public class UserManager {
    
    private List<User> users;
    private HashMap<String, User> userMap;
    
    /**
     * Constructor initializes the user collections
     */
    public UserManager() {
        this.users = new ArrayList<>();
        this.userMap = new HashMap<>();
    }
    
    /**
     * Adds a new user to the system
     * @param user The user to add
     * @return true if successful, false otherwise
     */
    public boolean addUser(User user) {
        if (user == null || user.getId() == null) {
            return false;
        }
        
        if (userMap.containsKey(user.getId())) {
            return false; // User already exists
        }
        
        users.add(user);
        userMap.put(user.getId(), user);
        return true;
    }
    
    /**
     * Retrieves a user by their ID
     * @param userId The user's ID
     * @return The user if found, null otherwise
     */
    public User getUser(String userId) {
        return userMap.get(userId);
    }
    
    /**
     * Removes a user from the system
     * @param userId The ID of the user to remove
     * @return true if removed, false if not found
     */
    public boolean removeUser(String userId) {
        User user = userMap.remove(userId);
        if (user != null) {
            users.remove(user);
            return true;
        }
        return false;
    }
    
    /**
     * Gets all users in the system
     * @return List of all users
     */
    public List<User> getAllUsers() {
        return new ArrayList<>(users);
    }
}
"""

# Save to file
os.makedirs('sample_java', exist_ok=True)
with open('sample_java/UserManager.java', 'w') as f:
    f.write(sample_java_code)

print("✓ Sample Java file created: sample_java/UserManager.java")
print(f"File size: {len(sample_java_code)} characters")

✓ Sample Java file created: sample_java/UserManager.java
File size: 1673 characters


## Step 6: Initialize the Chunker and Process the Java Code

Now we'll:
1. Create a chunker instance with our desired parameters
2. Process the Java code
3. Examine the resulting chunks

In [18]:
# Initialize the chunker
chunker = JavaCodeChunker(
    max_chunk_size=1000,   # Maximum 1000 characters per chunk
    chunk_overlap=200,     # 200 characters overlap between chunks
    min_chunk_size=100     # Merge chunks smaller than 100 characters
)

# Chunk the Java code
chunks = chunker.chunk_code(sample_java_code, file_path='sample_java/UserManager.java')

print(f"✓ Code chunked successfully!")
print(f"Total chunks created: {len(chunks)}")
print("\nChunk breakdown:")
for i, chunk in enumerate(chunks, 1):
    print(f"  {i}. {chunk}")

✓ JavaCodeChunker initialized with tree-sitter
✓ Code chunked successfully!
Total chunks created: 9

Chunk breakdown:
  1. CodeChunk(type=merged, lines=2-18, length=230)
  2. CodeChunk(type=method, lines=19-22, length=309)
  3. CodeChunk(type=class_member, lines=24-28, length=243)
  4. CodeChunk(type=method, lines=29-41, length=461)
  5. CodeChunk(type=class_member, lines=43-47, length=332)
  6. CodeChunk(type=method, lines=48-50, length=210)
  7. CodeChunk(type=class_member, lines=52-56, length=229)
  8. CodeChunk(type=method, lines=57-64, length=360)
  9. CodeChunk(type=merged, lines=66-72, length=360)


## Step 7: Examine Individual Chunks

Let's look at what each chunk contains to understand how the code was split.

In [19]:
# Display each chunk with details
for i, chunk in enumerate(chunks, 1):
    print(f"\n{'='*80}")
    print(f"CHUNK {i}/{len(chunks)}")
    print(f"{'='*80}")
    print(f"Type: {chunk.chunk_type}")
    print(f"Lines: {chunk.start_line}-{chunk.end_line}")
    print(f"Length: {len(chunk.content)} characters")
    print(f"Metadata: {json.dumps(chunk.metadata, indent=2)}")
    print(f"\nContent Preview (first 300 chars):")
    print(f"{'-'*80}")
    print(chunk.content[:300] + ("..." if len(chunk.content) > 300 else ""))
    print(f"{'-'*80}")


CHUNK 1/9
Type: merged
Lines: 2-18
Length: 230 characters
Metadata: {
  "file_path": "sample_java/UserManager.java",
  "node_type": "block_comment",
  "length": 63,
  "class_name": "UserManager"
}

Content Preview (first 300 chars):
--------------------------------------------------------------------------------
package com.example.app;
import java.util.List;
import java.util.ArrayList;
import java.util.HashMap;
private List<User> users;
private HashMap<String, User> userMap;
/**
     * Constructor initializes the user collections
     */
--------------------------------------------------------------------------------

CHUNK 2/9
Type: method
Lines: 19-22
Length: 309 characters
Metadata: {
  "file_path": "sample_java/UserManager.java",
  "node_type": "constructor_declaration",
  "length": 108,
  "class_name": "UserManager",
  "method_name": "UserManager",
  "has_overlap": true
}

Content Preview (first 300 chars):
------------------------------------------------------------------------

## Step 8: Set Up ChromaDB

Now we'll create a ChromaDB collection to store our code chunks.

**ChromaDB Features:**
- Automatically generates embeddings for text
- Supports semantic search (finding similar code)
- Stores metadata alongside embeddings
- Persistent storage on disk

In [43]:
import numpy as np
from sentence_transformers import SentenceTransformer
from datetime import datetime

class ChromaDBManager:
    """
    Manager for storing and retrieving Java code chunks in ChromaDB.
    """
    
    def __init__(
        self,
        collection_name: str = "java_code",
        persist_directory: str = "./chroma_db",
        model_name: str = "all-MiniLM-L6-v2",
        embedding_directory: str = "./embeddings",
        save_embeddings: bool = True
    ):
        
        
        """
        Initialize ChromaDB manager.
        
        Args:
            collection_name: Name of the collection to store chunks
            persist_directory: Directory where ChromaDB stores data
        """
        
        self.save_embeddings = save_embeddings
        # Setup embedding directories
        self.embedding_dir = Path(embedding_directory)
        self.documents_dir = self.embedding_dir / "documents"
        self.queries_dir = self.embedding_dir / "queries"
        
        if self.save_embeddings:
            self.documents_dir.mkdir(parents=True, exist_ok=True)
            self.queries_dir.mkdir(parents=True, exist_ok=True)
            print(f"📁 Embedding directories created:")
            print(f"   Documents: {self.documents_dir}")
            print(f"   Queries: {self.queries_dir}")


        
        
        print(f"Step 1: Loading embedding model '{model_name}'...")
        self.embedding_model = SentenceTransformer(model_name)
        print(f"  ✓ Model loaded")
        print(f"  • Model dimensions: {self.embedding_model.get_sentence_embedding_dimension()}")
        print(f"  • Max sequence length: {self.embedding_model.max_seq_length}")

        # Create persistent client
        self.client = chromadb.PersistentClient(path=persist_directory)
        
        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            metadata={"description": "Java code chunks with semantic embeddings"}
        )
        
        print(f"✓ ChromaDB initialized")
        print(f"  Collection: {collection_name}")
        print(f"  Storage: {persist_directory}")
        print(f"  Existing chunks: {self.collection.count()}")
    
    def generate_embedding(self, text: str,is_query: bool = False) -> np.ndarray:
        """
        Generate embedding for a single text.
            
        This is what ChromaDB does automatically behind the scenes!
            """
        # Tokenize and encode
        embedding = self.embedding_model.encode(text, show_progress_bar=False)
        
        # Save to disk if enabled
        if self.save_embeddings:
            self._save_embedding_to_disk(text, embedding, is_query)   
        
        return embedding

    def add_chunks(self, chunks: List[CodeChunk], batch_size: int = 100):
        """
        Add code chunks to ChromaDB.
        
        Process:
        1. Extract text content from chunks
        2. Generate unique IDs for each chunk
        3. Prepare metadata
        4. Add to ChromaDB (which auto-generates embeddings)
        """
        if not chunks:
            print("No chunks to add")
            return
        
        documents = []
        metadatas = []
        ids = []
        embeddings = []
        for i, chunk in enumerate(chunks):
            # Use chunk content as document
            documents.append(chunk.content)
            content = chunk.content
            # Prepare metadata (ChromaDB requires string values)
            metadata = {
                'chunk_type': chunk.chunk_type,
                'start_line': str(chunk.start_line),
                'end_line': str(chunk.end_line),
                'file_path': chunk.metadata.get('file_path', ''),
                'class_name': chunk.metadata.get('class_name', ''),
                'method_name': chunk.metadata.get('method_name', '')
            }
            metadatas.append(metadata)
            
            print(f"  Generating embedding...")
            
            embedding = self.generate_embedding(content)
            embeddings.append(embedding.tolist())
            
            print(f"  ✓ Embedding generated: {len(embedding)} dimensions")
            print(f"    First 5 values: {embedding[:5]}")
            print(f"    Norm (magnitude): {np.linalg.norm(embedding):.4f}")
            print()
            # Generate unique ID
            chunk_id = f"{chunk.metadata.get('file_path', 'unknown')}_{chunk.chunk_type}_{i}"
            ids.append(chunk_id)
        
        # Add to ChromaDB in batches
        for i in range(0, len(documents), batch_size):
            batch_docs = documents[i:i+batch_size]
            batch_metas = metadatas[i:i+batch_size]
            batch_ids = ids[i:i+batch_size]
            
            self.collection.add(
                documents=batch_docs,
                embeddings=embeddings,
                metadatas=batch_metas,
                ids=batch_ids
            )
        
        print(f"✓ Added {len(chunks)} chunks to ChromaDB")
        print(f"  Total chunks in collection: {self.collection.count()}")
    
    def search(self, query: str, n_results: int = 5) -> Dict:
        """
        Search for similar code chunks using semantic similarity.
        
        Args:
            query: Search query (e.g., "method to add user")
            n_results: Number of results to return
            
        Returns:
            Dictionary with documents, metadatas, and distances
        """
        query_embedding = self.generate_embedding(query, is_query=True)

        results = self.collection.query(
            query_embeddings=[query_embedding.tolist()],  # ← WE provide query embedding!
            n_results=n_results
        )
        
        return results
    
    def get_all_chunks(self) -> Dict:
        """Retrieve all chunks from the collection."""
        return self.collection.get()
    
    def delete_collection(self):
        """Delete the entire collection."""
        self.client.delete_collection(self.collection.name)
        print(f"✓ Deleted collection: {self.collection.name}")

    def _save_embedding_to_disk(self, text: str, embedding: np.ndarray, is_query: bool):
        """
        Save embedding and associated text to disk.
        
        File format:
        {
            "text": "original text",
            "embedding": [0.123, -0.456, ...],
            "metadata": {
                "timestamp": "2024-01-31T10:30:00",
                "type": "query" or "document",
                "dimensions": 384,
                "norm": 12.34,
                "model": "all-MiniLM-L6-v2"
            }
        }
        """
        # Choose directory
        save_dir = self.queries_dir if is_query else self.documents_dir
        
        # Generate filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        prefix = "query" if is_query else "doc"
        
        # Create safe filename from text
        safe_text = "".join(c if c.isalnum() else "_" for c in text[:50])
        filename = f"{prefix}_{timestamp}_{safe_text}.json"
        filepath = save_dir / filename
        
        # Prepare data
        data = {
            "text": text,
            "embedding": embedding.tolist(),
            "metadata": {
                "timestamp": datetime.now().isoformat(),
                "type": "query" if is_query else "document",
                "dimensions": len(embedding),
                "norm": float(np.linalg.norm(embedding)),
                "model": self.embedding_model._model_card_vars.get("model_name", "unknown"),
                "text_length": len(text),
                "word_count": len(text.split())
            }
        }
        
        # Save to file
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        return filepath

    def load_embedding_from_disk(self, filepath: str) -> Dict:
        """
        Load a saved embedding from disk.
        
        Returns:
            Dictionary with text, embedding, and metadata
        """
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Convert embedding back to numpy array
        data['embedding'] = np.array(data['embedding'])
        return data
print("✓ ChromaDBManager class defined")

✓ ChromaDBManager class defined


## Step 9: Initialize ChromaDB and Store Chunks

Now we'll:
1. Create a ChromaDB instance
2. Store our Java code chunks
3. ChromaDB will automatically generate embeddings for each chunk

In [41]:
# Initialize ChromaDB
db_manager = ChromaDBManager(
    collection_name="java_code_examples",
    persist_directory="./my_chroma_db"
)

# Add chunks to database
db_manager.add_chunks(chunks)

📁 Embedding directories created:
   Documents: embeddings\documents
   Queries: embeddings\queries
Step 1: Loading embedding model 'all-MiniLM-L6-v2'...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 221.13it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


  ✓ Model loaded
  • Model dimensions: 384
  • Max sequence length: 256
✓ ChromaDB initialized
  Collection: java_code_examples
  Storage: ./my_chroma_db
  Existing chunks: 18
  Generating embedding...
  ✓ Embedding generated: 384 dimensions
    First 5 values: [ 0.03048645  0.07178549 -0.05774655 -0.09416531 -0.0847486 ]
    Norm (magnitude): 1.0000

  Generating embedding...
  ✓ Embedding generated: 384 dimensions
    First 5 values: [ 0.0595382   0.08578359 -0.03420369 -0.08280393 -0.09614842]
    Norm (magnitude): 1.0000

  Generating embedding...
  ✓ Embedding generated: 384 dimensions
    First 5 values: [ 0.02899302  0.04080064 -0.05908169 -0.04000561 -0.06272503]
    Norm (magnitude): 1.0000

  Generating embedding...
  ✓ Embedding generated: 384 dimensions
    First 5 values: [-0.00962093 -0.00122156 -0.04424215 -0.05495535 -0.08957515]
    Norm (magnitude): 1.0000

  Generating embedding...
  ✓ Embedding generated: 384 dimensions
    First 5 values: [ 0.00383892  0.06987098 -

## Step 10: Test Semantic Search

Now let's test the search functionality!

ChromaDB uses **semantic similarity** - it understands the *meaning* of your query,
not just keyword matching.

For example:
- Query: "add user" → Should find the `addUser` method
- Query: "remove user" → Should find the `removeUser` method
- Query: "initialize collections" → Should find the constructor

In [42]:
# Test search with different queries
test_queries = [
    "method to add a new user",
    "remove user from system",
    "get all users",
    "constructor initialization"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"SEARCH: '{query}'")
    print(f"{'='*80}")
    
    results = db_manager.search(query, n_results=3)
    
    for i, (doc, meta, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ), 1):
        print(f"\nResult {i} (similarity: {1 - distance:.3f}):")
        print(f"  Type: {meta['chunk_type']}")
        print(f"  Class: {meta.get('class_name', 'N/A')}")
        print(f"  Method: {meta.get('method_name', 'N/A')}")
        print(f"  Lines: {meta['start_line']}-{meta['end_line']}")
        print(f"  \nCode Preview:")
        print(f"  {'-'*76}")
        preview = doc[:200].replace('\n', '\n  ')
        print(f"  {preview}...")
        print(f"  {'-'*76}")


SEARCH: 'method to add a new user'

Result 1 (similarity: 0.287):
  Type: method
  Class: UserManager
  Method: addUser
  Lines: 29-41
  
Code Preview:
  ----------------------------------------------------------------------------
  /**
       * Adds a new user to the system
       * @param user The user to add
       * @return true if successful, false otherwise
       */
  public boolean addUser(User user) {
          if (user == null || u...
  ----------------------------------------------------------------------------

Result 2 (similarity: 0.287):
  Type: method
  Class: UserManager
  Method: addUser
  Lines: 29-41
  
Code Preview:
  ----------------------------------------------------------------------------
  /**
       * Adds a new user to the system
       * @param user The user to add
       * @return true if successful, false otherwise
       */
  public boolean addUser(User user) {
          if (user == null || u...
  -------------------------------------------------------

## Step 11: Process Multiple Java Files

Let's create a utility function to process an entire directory of Java files.

In [23]:
def process_java_directory(directory_path: str, chunker: JavaCodeChunker) -> List[CodeChunk]:
    """
    Process all Java files in a directory.
    
    Args:
        directory_path: Path to directory containing Java files
        chunker: JavaCodeChunker instance
        
    Returns:
        List of all code chunks from all files
    """
    all_chunks = []
    java_files = list(Path(directory_path).rglob('*.java'))
    
    print(f"Found {len(java_files)} Java files in {directory_path}")
    
    for java_file in java_files:
        print(f"  Processing: {java_file}")
        
        # Read file
        with open(java_file, 'r', encoding='utf-8') as f:
            code = f.read()
        
        # Chunk the code
        file_chunks = chunker.chunk_code(code, file_path=str(java_file))
        all_chunks.extend(file_chunks)
        
        print(f"    → Created {len(file_chunks)} chunks")
    
    print(f"\n✓ Total chunks from all files: {len(all_chunks)}")
    return all_chunks

# Example: Process the sample directory
all_chunks = process_java_directory('sample_java', chunker)

# Add all chunks to ChromaDB
db_manager.add_chunks(all_chunks)

Found 1 Java files in sample_java
  Processing: sample_java\UserManager.java
    → Created 9 chunks

✓ Total chunks from all files: 9
✓ Added 9 chunks to ChromaDB
  Total chunks in collection: 18


## Step 12: Advanced Search Examples

Let's try some more sophisticated searches to demonstrate the power of semantic search.

In [24]:
def pretty_print_search_results(query: str, n_results: int = 3):
    """
    Perform search and display results in a readable format.
    """
    print(f"\n{'🔍 SEARCH QUERY':-^80}")
    print(f"  {query}")
    print(f"{'':-^80}\n")
    
    results = db_manager.search(query, n_results=n_results)
    
    if not results['documents'][0]:
        print("  No results found.")
        return
    
    for i, (doc, meta, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ), 1):
        similarity_score = (1 - distance) * 100
        
        print(f"  📄 Result #{i} | Similarity: {similarity_score:.1f}%")
        print(f"  {'─' * 76}")
        print(f"    📍 Location: {meta['file_path']}")
        print(f"    📊 Type: {meta['chunk_type']}")
        print(f"    📝 Lines: {meta['start_line']}-{meta['end_line']}")
        
        if meta.get('class_name'):
            print(f"    🏛️  Class: {meta['class_name']}")
        if meta.get('method_name'):
            print(f"    ⚡ Method: {meta['method_name']}")
        
        print(f"\n    💻 Code:")
        code_lines = doc.split('\n')[:10]  # First 10 lines
        for line in code_lines:
            print(f"       {line}")
        if len(doc.split('\n')) > 10:
            print(f"       ...")
        print()

# Try various search queries
search_examples = [
    "How do I check if a user already exists?",
    "Code that validates user input",
    "Return a list of users",
    "HashMap implementation"
]

for query in search_examples:
    pretty_print_search_results(query, n_results=2)


---------------------------------🔍 SEARCH QUERY---------------------------------
  How do I check if a user already exists?
--------------------------------------------------------------------------------

  📄 Result #1 | Similarity: 24.1%
  ────────────────────────────────────────────────────────────────────────────
    📍 Location: sample_java/UserManager.java
    📊 Type: method
    📝 Lines: 29-41
    🏛️  Class: UserManager
    ⚡ Method: addUser

    💻 Code:
       /**
            * Adds a new user to the system
            * @param user The user to add
            * @return true if successful, false otherwise
            */
       public boolean addUser(User user) {
               if (user == null || user.getId() == null) {
                   return false;
               }
       
       ...

  📄 Result #2 | Similarity: 24.1%
  ────────────────────────────────────────────────────────────────────────────
    📍 Location: sample_java\UserManager.java
    📊 Type: method
    📝 Lines: 29-

## Step 13: Retrieve Statistics and Metadata

Let's examine what we've stored in ChromaDB.

In [25]:
# Get collection statistics
total_chunks = db_manager.collection.count()
all_data = db_manager.get_all_chunks()

print(f"📊 ChromaDB Statistics")
print(f"{'='*80}")
print(f"Total Chunks: {total_chunks}")
print(f"\nChunk Type Distribution:")

# Count chunk types
from collections import Counter
chunk_types = Counter(meta['chunk_type'] for meta in all_data['metadatas'])

for chunk_type, count in chunk_types.most_common():
    bar = '█' * (count * 2)
    print(f"  {chunk_type:15s} | {bar} {count}")

# Calculate storage statistics
total_chars = sum(len(doc) for doc in all_data['documents'])
avg_chunk_size = total_chars / total_chunks if total_chunks > 0 else 0

print(f"\nStorage Statistics:")
print(f"  Total characters: {total_chars:,}")
print(f"  Average chunk size: {avg_chunk_size:.0f} characters")
print(f"  Estimated embeddings size: ~{total_chunks * 384 * 4 / 1024 / 1024:.2f} MB")
print(f"  (assuming 384-dimensional embeddings with 4 bytes per dimension)")

📊 ChromaDB Statistics
Total Chunks: 18

Chunk Type Distribution:
  method          | ████████████████ 8
  class_member    | ████████████ 6
  merged          | ████████ 4

Storage Statistics:
  Total characters: 5,468
  Average chunk size: 304 characters
  Estimated embeddings size: ~0.03 MB
  (assuming 384-dimensional embeddings with 4 bytes per dimension)


## Step 14: Complete Workflow Function

Here's a complete function that ties everything together for easy reuse.

In [26]:
def complete_java_to_chromadb_workflow(
    java_directory: str,
    collection_name: str = "java_code",
    max_chunk_size: int = 1000,
    chunk_overlap: int = 200,
    persist_directory: str = "./chroma_db"
):
    """
    Complete workflow: Java code → Chunks → ChromaDB
    
    Steps:
    1. Initialize chunker with parameters
    2. Process all Java files in directory
    3. Create chunks using AST-based splitting
    4. Store chunks in ChromaDB with embeddings
    5. Return database manager for querying
    
    Args:
        java_directory: Path to directory with Java files
        collection_name: Name for ChromaDB collection
        max_chunk_size: Maximum characters per chunk
        chunk_overlap: Overlapping characters between chunks
        persist_directory: Where to store ChromaDB data
        
    Returns:
        ChromaDBManager instance for searching
    """
    
    print(f"\n{'🚀 STARTING JAVA CODE PROCESSING WORKFLOW':-^80}\n")
    
    # Step 1: Initialize chunker
    print("Step 1: Initializing Java Code Chunker...")
    chunker = JavaCodeChunker(
        max_chunk_size=max_chunk_size,
        chunk_overlap=chunk_overlap,
        min_chunk_size=100
    )
    
    # Step 2: Process Java files
    print(f"\nStep 2: Processing Java files from '{java_directory}'...")
    chunks = process_java_directory(java_directory, chunker)
    
    if not chunks:
        print("❌ No chunks created. Check if Java files exist in the directory.")
        return None
    
    # Step 3: Initialize ChromaDB
    print(f"\nStep 3: Initializing ChromaDB...")
    db_manager = ChromaDBManager(
        collection_name=collection_name,
        persist_directory=persist_directory
    )
    
    # Step 4: Store chunks
    print(f"\nStep 4: Storing chunks in ChromaDB...")
    db_manager.add_chunks(chunks)
    
    print(f"\n{'✅ WORKFLOW COMPLETED SUCCESSFULLY':-^80}\n")
    print(f"📊 Summary:")
    print(f"  • Processed: {len(list(Path(java_directory).rglob('*.java')))} Java files")
    print(f"  • Created: {len(chunks)} code chunks")
    print(f"  • Stored in: {collection_name} collection")
    print(f"  • Database: {persist_directory}")
    print(f"\n💡 You can now search using: db_manager.search('your query')\n")
    
    return db_manager

# Example usage
print("\n" + "="*80)
print("EXAMPLE: Running complete workflow")
print("="*80)

db = complete_java_to_chromadb_workflow(
    java_directory='sample_java',
    collection_name='my_java_codebase',
    max_chunk_size=1000,
    chunk_overlap=200
)


EXAMPLE: Running complete workflow

--------------------🚀 STARTING JAVA CODE PROCESSING WORKFLOW--------------------

Step 1: Initializing Java Code Chunker...
✓ JavaCodeChunker initialized with tree-sitter

Step 2: Processing Java files from 'sample_java'...
Found 1 Java files in sample_java
  Processing: sample_java\UserManager.java
    → Created 9 chunks

✓ Total chunks from all files: 9

Step 3: Initializing ChromaDB...
✓ ChromaDB initialized
  Collection: my_java_codebase
  Storage: ./chroma_db
  Existing chunks: 0

Step 4: Storing chunks in ChromaDB...
✓ Added 9 chunks to ChromaDB
  Total chunks in collection: 9

-----------------------✅ WORKFLOW COMPLETED SUCCESSFULLY------------------------

📊 Summary:
  • Processed: 1 Java files
  • Created: 9 code chunks
  • Stored in: my_java_codebase collection
  • Database: ./chroma_db

💡 You can now search using: db_manager.search('your query')



## Step 15: Final Testing and Use Cases

Let's demonstrate practical use cases for the code search system.

In [27]:
# Use Case 1: Find code for a specific functionality
print("\n" + "="*80)
print("USE CASE 1: Finding specific functionality")
print("="*80)
pretty_print_search_results("How to validate if user exists before adding", n_results=2)

# Use Case 2: Find similar code patterns
print("\n" + "="*80)
print("USE CASE 2: Finding similar patterns")
print("="*80)
pretty_print_search_results("null checking and validation", n_results=2)

# Use Case 3: Find code by description
print("\n" + "="*80)
print("USE CASE 3: Natural language description")
print("="*80)
pretty_print_search_results("get user by their unique identifier", n_results=2)


USE CASE 1: Finding specific functionality

---------------------------------🔍 SEARCH QUERY---------------------------------
  How to validate if user exists before adding
--------------------------------------------------------------------------------

  📄 Result #1 | Similarity: 34.7%
  ────────────────────────────────────────────────────────────────────────────
    📍 Location: sample_java/UserManager.java
    📊 Type: method
    📝 Lines: 29-41
    🏛️  Class: UserManager
    ⚡ Method: addUser

    💻 Code:
       /**
            * Adds a new user to the system
            * @param user The user to add
            * @return true if successful, false otherwise
            */
       public boolean addUser(User user) {
               if (user == null || user.getId() == null) {
                   return false;
               }
       
       ...

  📄 Result #2 | Similarity: 34.7%
  ────────────────────────────────────────────────────────────────────────────
    📍 Location: sample_java\User

## Summary and Key Takeaways

### What We Accomplished:

1. **✅ Tree-Based Code Parsing**
   - Used tree-sitter to parse Java code into AST
   - Intelligently split code at semantic boundaries
   - Preserved class and method structures

2. **✅ Smart Chunking Strategy**
   - Configurable chunk sizes
   - Overlap between chunks for context
   - Merging of small chunks
   - Metadata extraction (class names, method names, etc.)

3. **✅ ChromaDB Integration**
   - Automatic embedding generation
   - Semantic search capabilities
   - Persistent storage
   - Metadata filtering

### Benefits of This Approach:

- **🎯 Semantic Understanding**: Find code by meaning, not just keywords
- **🏗️ Structure Preservation**: Chunks are complete, meaningful units
- **🔍 Better Retrieval**: Context-aware search results
- **📊 Rich Metadata**: Filter by file, class, method, type, etc.
- **💾 Scalable**: Works with large codebases

### Next Steps:

1. **Scale to your codebase**: Point to your Java project directory
2. **Customize chunking**: Adjust chunk sizes based on your needs
3. **Enhance search**: Add filtering by file, class, or method
4. **Build applications**: RAG systems, code assistants, documentation generators
5. **Extend to other languages**: Adapt the chunker for Python, JavaScript, etc.

### Performance Tips:

- For large codebases, process files in batches
- Adjust `max_chunk_size` based on your use case
- Use metadata filters to narrow search scope
- Consider using a more powerful embedding model for better search quality

---

**🎉 Congratulations! You now have a working code search system using tree-based chunking and ChromaDB!**